In [ ]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [ ]:
import os
import re
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_hub as hub
from sklearn import preprocessing
from sklearn.neighbors import NearestNeighbors

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
root_path = 'gdrive/Md Drive'
os.chdir('gdrive/My Drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [ ]:
#ADJUST THESE
labelled_file_path = 'LabeledData.csv'
unlabelled1_path = 'IC-output04162020-153940.csv'
unlabelled1_output_path = 'IC_output.csv'
#add more csv's to be scored
#add path to output csv
unlabelled2_path = 'IC-output04162020-153940-small.csv'
unlabelled2_output_path = 'IC-small_output.csv'

#number of neighbors to search
k = 3

In [ ]:
#read labelled data
l_df = pd.read_csv(labelled_file_path, encoding='utf8')
l_df.head()

,Name,OCR File Name if exists,Notice of Assignment required,Supplier Consent required,Exception to consent present,Extracted Assignment Language,Extraction Accuracy
0,Agile Transformation Alliance Award 030215.pdf,Agile Transformation Alliance Award 030215_OCR...,N,N,NaN,No Assignment Extracted,NaN
1,Alfresco MSLA Award 100115 fe.pdf,Alfresco MSLA Award 100115 fe_OCR.pdf,N,Y,Y,11.2 Assignment. This Agreement is assignable...,NaN
2,AlgoSec EULA revised fe.pdf,AlgoSec EULA revised fe_OCR.pdf,N,Y,Y,No Assignment Extracted,NaN
3,Allen Systems_SW_MSLA_12202002 0.pdf,Allen Systems_SW_MSLA_12202002 0.pdf,N,Y,Y,(a) Assignment - Notwithstanding anything to t...,NaN
4,American Airlines 121120.pdf,American Airlines 121120_OCR.pdf,Y,N,NaN,No Assignment Extracted,NaN


In [ ]:
#CLEAN LABELLED DATA- add rules as necessary

#remove rows with no file name
l_df.iloc[:]['OCR File Name if exists'].fillna('Empty', inplace=True)
c1 = l_df['OCR File Name if exists'] != 'Empty'
l_df = l_df[c1]

#remove rows with empty extractions
l_df.iloc[:]['Extracted Assignment Language'].fillna('No Assignment Extracted', inplace=True)

#remove rows where extraction = 'No Assignment Extracted'
c2 = l_df['Extracted Assignment Language'] != 'No Assignment Extracted'
l_df = l_df[c2]

l_df = l_df.reset_index(drop=True)

In [ ]:
#view cleaned
l_df.head()

,Name,OCR File Name if exists,Notice of Assignment required,Supplier Consent required,Exception to consent present,Extracted Assignment Language,Extraction Accuracy
0,Alfresco MSLA Award 100115 fe.pdf,Alfresco MSLA Award 100115 fe_OCR.pdf,N,Y,Y,11.2 Assignment. This Agreement is assignable...,NaN
1,Allen Systems_SW_MSLA_12202002 0.pdf,Allen Systems_SW_MSLA_12202002 0.pdf,N,Y,Y,(a) Assignment - Notwithstanding anything to t...,NaN
2,Appian VASP with Referral Appendix 091416 FE.pdf,Appian VASP with Referral Appendix 091416 FE_O...,N,Y,N,16.4 Assignment: This Agreement shall be bindi...,NaN
3,APS-CSRA_VAR_11272017 Signed.pdf,APS-CSRA_VAR_11272017 Signed_OCR.pdf,N,Y,0,11.5. Assignment. This Agreement will be bind...,NaN
4,AWS Enterprise Customer Agmt Award 092415.pdf,AWS Enterprise Customer Agmt Award 092415_OCR.pdf,N,Y,Y,12.8 Assignment. Neither party may assign or o...,NaN


In [ ]:
url = "https://tfhub.dev/google/elmo/2"
embed = hub.Module(url)

In [ ]:
def cleanExtracts(sin):
    s = str(sin)
    regex = re.compile(r'[\n\r\t,"Äã]')  #remove CR, LF, Tabs, Commas, and quotes
    s = regex.sub("", s)
    regex = re.compile(r'[¬†]')
    s = regex.sub(' ', s)
    s1 = s[:5]
    s2 = s[5:]
    s1 = re.sub(r"\d+", "", s1)
    s1 = s1.replace('.','')
    s = s1.split(' ',1)[-1] + s2
    return s.strip()

In [ ]:
def defineEmbeddings(start, end, extractions):    
    embeddings = embed(extractions[start:end],signature="default",as_dict=True)["default"]
    return embeddings

In [ ]:
def scoreExtractions (embeddings):
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        sess.run(tf.tables_initializer())
        x = sess.run(embeddings)
    return x

In [ ]:
def create_points(extractions, bs):
    batch_size = bs
    scores = []
    for start in range(0,len(extractions), batch_size):
        end = start+batch_size
        embeddings = defineEmbeddings(start, end, extractions)
        x = scoreExtractions(embeddings)
        scores.extend(x)
    return scores

In [ ]:
%%time
#create embeddings for labelled data
labelled_extractions = [cleanExtracts(ex) for ex in list(l_df['Extracted Assignment Language'])]
labelled_scores = create_points(labelled_extractions, 1)

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


...

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


CPU times: user 2min 30s, sys: 12 s, total: 2min 42s
Wall time: 2min 46s


In [ ]:
#create nearest neighbors model from labelled data
X = np.array(labelled_scores)
nbrs = NearestNeighbors(n_neighbors=k, algorithm='ball_tree').fit(X)

In [ ]:
#SCORE UNLABELLED DATA

In [ ]:
def record_mult_extracts(ul_df):
    fn_inds = dict()
    for i in range(ul_df.shape[0]):
        fn = ul_df.iloc[i]['File_Name']
        try:
            inds = fn_inds[fn]
            inds.append(i)
            fn_inds[fn] = inds
        except:
            fn_inds[fn] = [i]
    return fn_inds

In [ ]:
def determine_class(ns_scoring):
    count_y = 0
    total = len(ns_scoring)
    threshold = 0.5
    for s in ns_scoring:
        if s == 'Y':
            count_y += 1
    rat = count_y/total
    if rat == 0.5:
        return 'N/A'
    elif rat >= threshold:
        return 'Y'
    else:
        return 'N'

In [ ]:
def determine_classes(fn_inds, unlabelled_scores, classes):
    new_scoring = []
    for fn in fn_inds.keys():
        inds = fn_inds[fn]
        x = []
        for i in inds:
            x.append(unlabelled_scores[i])
        _, indices = nbrs.kneighbors(x)
        indices = np.array(indices)
        indices = indices.reshape(-1)
        i_scoring = []
        for j in range(len(classes)):
            ns_scoring_classj = []
            for k in indices:
                ns_scoring_classj.append(l_df.iloc[k][classes[j]])
            i_scoring.append(determine_class(ns_scoring_classj))
        new_scoring.append(i_scoring)
    return new_scoring

In [ ]:
def score_unlabelled(ul_input_path, ul_output_path):
    ul_df = pd.read_csv(ul_input_path)
    ul_df = ul_df.drop(['Unnamed: 0'], axis=1)
    fn_inds = record_mult_extracts(ul_df)

    #create embeddings for unlabelled data
    unlabelled_extractions = [cleanExtracts(ex) for ex in list(ul_df['Extracted'])]
    unlabelled_scores = create_points(unlabelled_extractions, 1)

    #determine classes for unlabelled data
    classes = ['Notice of Assignment required', 'Supplier Consent required', 'Exception to consent present']
    new_scoring = determine_classes(fn_inds, unlabelled_scores, classes)

    #save classifications to csv
    new_df = pd.DataFrame()
    new_df['Name'] = list(fn_inds.keys())
    new_df[classes] = pd.DataFrame(new_scoring)
    new_df.to_csv(ul_output_path)

In [ ]:
#CALL score_unlabelled() ON NEW DATA
score_unlabelled(unlabelled2_path, unlabelled2_output_path)
#add more

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


...

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore
